# Model

## Logistic regression

In [ ]:
# Load packages

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# Load Data 


## Non imputed Data 
df_train = pd.read_parquet("../data/processed/merged/non_imputed/train.parquet")
df_val = pd.read_parquet("../data/processed/merged/non_imputed/validation.parquet")

df_train.drop(columns="pre_relief_refi_loan_seq_number", inplace=True)
df_val.drop(columns="pre_relief_refi_loan_seq_number", inplace=True)

## Imputed data
df_train_imp = pd.read_parquet("../data/processed/merged/imputed/train.parquet")
df_val_imp = pd.read_parquet("../data/processed/merged/imputed/validation.parquet")

In [ ]:
df_train_imp["defa"]

In [ ]:
# Corrélation cell 



# MAP each column into a data types categories (to better select columns)

from collections import defaultdict

dict_types = {
 'credit_score': 'continuous',
 'first_payment_date': 'date',
 'first_time_homebuyer_flag': 'bool',            # Y/N, 9 -> NA
 'maturity_date': 'date',
 'msa_md': 'category',
 'mi_percent': 'continuous',                     # pas catégoriel
 'number_of_units': 'category',                  # 1–4, 99 -> NA (caté OK)
 'occupancy_status': 'category',
 'original_cltv': 'continuous',
 'original_dti': 'continuous',
 'original_upb': 'continuous',
 'original_ltv': 'continuous',
 'original_interest_rate': 'continuous',
 'channel': 'category',
 'ppm_flag': 'bool',                             # Y/N
 'amortization_type': 'category',                # FRM/ARM
 'property_state': 'category',
 'property_type': 'category',
 'postal_code': 'category',
 'loan_sequence_number': 'id',
 'loan_purpose': 'category',
 'original_loan_term': 'integer',                # nb de mois, pas une caté
 'number_of_borrowers': 'integer',               # petit entier
 'seller_name': 'category',
 'servicer_name': 'category',
 'super_conforming_flag': 'bool',                # Y / vide
 'pre_relief_refi_loan_seq_number': 'id_ref',    # clé de ref, pas une caté
 'special_eligibility_program': 'category',      # H/F/R/9
 'relief_refinance_indicator': 'bool',           # Y / vide
 'property_valuation_method': 'category',        # codes 1–4, 9 -> NA
 'interest_only_indicator': 'bool',              # Y/N
 'mi_cancellation_indicator': 'category',        # Y/N/7/9
 'default_24m': 'bool',
 'vintage': 'date'                               # ex. YYYY-Q à partir de FIRST_PAYMENT_DATE
}

# inversion type -> liste de colonnes
_dict_types_col = defaultdict(list)
for col, t in dict_types.items():
    _dict_types_col[t].append(col)

dict_types_col = dict(_dict_types_col)  # (optionnel) convertir en dict standard

cont_cols  = dict_types_col.get('continuous', [])
ord_cols   = dict_types_col.get('integer', [])      # ici: original_loan_term, number_of_borrowers (traités comme ordinal/binaire)
bools_cols = dict_types_col.get('bool', [])
cat_cols  = dict_types_col.get('category', []) + bools_cols



# Correlations cell

from scipy.stats import spearmanr, pointbiserialr, pearsonr, kendalltau
from scipy.stats import chi2_contingency
from itertools import combinations
import numpy as np

spearman_pairs  = {}
pearson_pairs   = {}
cramer_pairs    = {}
pb_pairs        = {}
kendall_pairs   = {}

# df_corr = df_train_imp
df_corr = df_train
# 1) Spearman : ordinal <-> continu (monotone)
for x in ord_cols:
    for y in cont_cols:
        s = df_corr[[x, y]].dropna()
        if s.empty: continue
        r, p = spearmanr(s[x], s[y])
        spearman_pairs[(x, y)] = (r, p)

# 2) Pearson : continu <-> continu (sans doublons ni diagonale)
for x, y in combinations(cont_cols, 2):
    s = df_corr[[x, y]].dropna()
    if s.empty: continue
    r, p = pearsonr(s[x], s[y])
    pearson_pairs[(x, y)] = (r, p)

# 3) Point-bisérial : continu <-> booléen binaire
bin_cols = [c for c in dict_types_col.get('bool', [])
            if set(df_corr[c].dropna().unique()).issubset({0,1,True,False})]
for x in cont_cols:
    for b in bin_cols:
        s = df_corr[[x, b]].dropna()
        if s.empty: continue
        r, p = pointbiserialr(s[x], s[b].astype(int))
        pb_pairs[(x, b)] = (r, p)   # <-- bug corrigé

# 4) Kendall tau-b : ordinal <-> ordinal (et en option ordinal <-> continu)
for x, y in combinations(ord_cols, 2):
    s = df_corr[[x, y]].dropna()
    if s.empty: continue
    tau, p = kendalltau(s[x], s[y], method="auto")  # tau-b avec correction des ties
    kendall_pairs[(x, y)] = (tau, p)

# Optionnel : Kendall pour ordinal <-> continu
for x in ord_cols:
    for y in cont_cols:
        s = df_corr[[x, y]].dropna()
        if s.empty: continue
        tau, p = kendalltau(s[x], s[y], method="auto")
        kendall_pairs[(x, y)] = (tau, p)

# 5) Cramér V corrigé : nominal/ordinal <-> nominal/ordinal
def cramers_v_corrected(x, y):
    tbl = pd.crosstab(x, y)
    if tbl.size == 0: 
        return np.nan
    chi2 = chi2_contingency(tbl, correction=False)[0]
    n = tbl.values.sum()
    r, k = tbl.shape
    phi2 = chi2 / n
    # Correction biais (Bergsma, 2013)
    phi2corr = max(0, phi2 - (k-1)*(r-1)/(n-1))
    rcorr = r - (r-1)**2/(n-1)
    kcorr = k - (k-1)**2/(n-1)
    denom = max(1e-12, min((kcorr-1), (rcorr-1)))
    return np.sqrt(phi2corr / denom)

cats = list(set(cat_cols) | set(ord_cols))  # traiter l'ordinal comme catégoriel ici
for a, b in combinations(cats, 2):
    s = df_corr[[a, b]].dropna()
    if s.empty: continue
    cramer_pairs[(a, b)] = cramers_v_corrected(s[a], s[b])


In [ ]:
# Shape the correlation into a long data frame


# ---------- 1) Fusion des dictionnaires en un tableau "long" ----------
def _dict_to_df(d, measure, has_p=True):
    rows = []
    for (x, y), val in d.items():
        # tolère (stat, p) ou (stat, p, n)
        if has_p:
            stat = val[0]
            pval = val[1]
            n = val[2] if (isinstance(val, (tuple, list)) and len(val) >= 3) else np.nan
        else:
            stat = val
            pval = np.nan
            n = np.nan
        rows.append({
            "var_x": x,
            "var_y": y,
            "measure": measure,
            "stat": float(stat),
            "p_value": float(pval) if pd.notna(pval) else np.nan,
            "n": float(n) if pd.notna(n) else np.nan,
            "abs_stat": abs(float(stat)),
        })
    return pd.DataFrame(rows)

dfs = []
if len(spearman_pairs): dfs.append(_dict_to_df(spearman_pairs, "spearman", has_p=True))
if len(pearson_pairs):  dfs.append(_dict_to_df(pearson_pairs,  "pearson",  has_p=True))
if len(pb_pairs):       dfs.append(_dict_to_df(pb_pairs,       "pointbiserial", has_p=True))
if len(kendall_pairs):  dfs.append(_dict_to_df(kendall_pairs,  "kendall_tau", has_p=True))
if len(cramer_pairs):   dfs.append(_dict_to_df(cramer_pairs,   "cramers_v", has_p=False))

assocs_long = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame(
    columns=["var_x","var_y","measure","stat","p_value","n","abs_stat"]
)

# ---------- 2) Ajout des types de variables (utile pour filtrer) ----------
def _type_of(v):
    if v in bin_cols:  return "bin"
    if v in cont_cols: return "cont"
    if v in ord_cols:  return "ord"
    if v in cat_cols:  return "cat"
    return "unknown"

assocs_long["type_x"] = assocs_long["var_x"].map(_type_of)
assocs_long["type_y"] = assocs_long["var_y"].map(_type_of)

# ---------- 3) Tri + étoiles de significativité + q-values (FDR BH) ----------
def _stars(p):
    if pd.isna(p): return ""
    return "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""

assocs_long["sig"] = assocs_long["p_value"].apply(_stars)

# FDR Benjamini–Hochberg (si statsmodels dispo)
try:
    from statsmodels.stats.multitest import multipletests
    m = assocs_long["p_value"].notna()
    assocs_long.loc[m, "q_value"] = multipletests(assocs_long.loc[m, "p_value"], method="fdr_bh")[1]
except Exception:
    assocs_long["q_value"] = np.nan

assocs_long = assocs_long.sort_values(["measure", "abs_stat"], ascending=[True, False]).reset_index(drop=True)

# ---------- 4) Exemples d’usages rapides ----------
# a) Top 20 associations par mesure (en valeur absolue)
top20_by_measure = (
    assocs_long.groupby("measure", group_keys=False)
               .apply(lambda g: g.nlargest(20, "abs_stat"))
               .reset_index(drop=True)
)

# b) Pivots pour heatmaps (ex: Pearson et Cramér V)
pearson_pivot = (assocs_long.query("measure == 'pearson'")
                              .pivot(index="var_x", columns="var_y", values="stat"))
cramers_pivot = (assocs_long.query("measure == 'cramers_v'")
                              .pivot(index="var_x", columns="var_y", values="stat"))

# ---------- 5) Export CSV ----------
assocs_long.to_csv("associations_long.csv", index=False)
top20_by_measure.to_csv("associations_top20_by_measure.csv", index=False)

print("OK • assocs_long :", assocs_long.shape, "| Sauvé: associations_long.csv")
assocs_long



In [ ]:
dict_threshold = {
    "spearman": 0.70,
    "pearson": 0.70,
    "pointbiserial": 0.70,
    "kendall_tau": 0.60,
    "cramers_v": 0.60
}

pair_var = pd.DataFrame(columns=assocs_long.columns)

for k, v in dict_threshold.items():
    mask = (
        (assocs_long["measure"] == k) &
        (assocs_long["stat"].notna()) &
        (assocs_long["stat"] >= v)
    )
    concat_df = assocs_long[mask]
    pair_var = pd.concat([pair_var, concat_df], ignore_index=True)

print(pair_var)


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss

# =========================
# 0) Préparation
# =========================
TARGET = "default_24m"

df = df_train_imp.copy()

# Règles métier de drop initial
initial_drop = set()

# IDs
initial_drop.add("loan_sequence_number")

# Dates -> on garde vintage par défaut
if "vintage" in df.columns:
    for c in ["first_payment_date", "maturity_date"]:
        if c in df.columns:
            initial_drop.add(c)

# CLTV vs LTV : garder LTV par défaut
if "original_cltv" in df.columns and "original_ltv" in df.columns:
    initial_drop.add("original_cltv")
    # si tu drop CLTV, l'indicateur de NA associé ne sert plus
    if "cltv_missing" in df.columns:
        initial_drop.add("cltv_missing")

# Géographie hiérarchique : drop postal_code et msa_md (par défaut)
for c in ["postal_code", "msa_md"]:
    if c in df.columns:
        initial_drop.add(c)

# Dérivés évidents
if "has_mi" in df.columns and "mi_percent" in df.columns:
    initial_drop.add("has_mi")
if "has_special_program" in df.columns:
    initial_drop.add("has_special_program")

# Applique drop initial
keep_cols = [c for c in df.columns if c not in initial_drop and c != TARGET]
X0 = df[keep_cols].copy()
y = df[TARGET].astype(int).values

# =========================
# 1) Scoring univarié par logit CV
# =========================
def score_univar_cv(X, y, n_splits=5, random_state=42):
    scores = {}
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for col in X.columns:
        xcol = X[[col]]
        # Détermine type
        is_num = xcol.dtypes.iloc[0].kind in "bifc"
        # Pipeline adapté
        if is_num:
            pipe = make_pipeline(
                StandardScaler(with_mean=False),  # robuste aux constantes/sparse
                LogisticRegression(solver="liblinear", max_iter=300)
            )
        else:
            pre = ColumnTransformer(
                [("oh", OneHotEncoder(drop="first", handle_unknown="ignore"), [col])],
                remainder="drop",
                verbose_feature_names_out=False
            )
            pipe = make_pipeline(pre, LogisticRegression(solver="liblinear", max_iter=300))

        # P(y=1) via CV (no leakage)
        p = cross_val_predict(pipe, xcol, y, cv=skf, method="predict_proba")[:, 1]
        # AUC / LogLoss
        auc = roc_auc_score(y, p)
        ll  = log_loss(y, p)
        # Taux de NA & cardinalité
        na_rate = xcol[col].isna().mean()
        card = xcol[col].nunique(dropna=True)

        scores[col] = {"auc": auc, "logloss": ll, "na_rate": na_rate, "cardinality": card, "is_numeric": is_num}

    return pd.DataFrame(scores).T.reset_index().rename(columns={"index": "variable"})

univar = score_univar_cv(X0, y)

# =========================
# 2) Recommandations de drop dans les paires corrélées
#     (à partir de ton pair_var calculé auparavant)
# =========================
# Hypothèse : pair_var contient var_x, var_y, measure, stat (seuil déjà appliqué)
# On merge les scores univariés pour comparer
pv = pair_var.copy()
pv = pv.merge(univar.add_prefix("x_"), left_on="var_x", right_on="x_variable", how="left")
pv = pv.merge(univar.add_prefix("y_"), left_on="var_y", right_on="y_variable", how="left")

def choose_drop(row, auc_eps=0.005, ll_eps=0.001):
    # Priorité 1 : AUC (plus haut = mieux)
    ax, ay = row["x_auc"], row["y_auc"]
    lx, ly = row["x_logloss"], row["y_logloss"]
    nx, ny = row["x_na_rate"], row["y_na_rate"]
    cx, cy = row["x_cardinality"], row["y_cardinality"]

    # Si AUC très proche, regarde logloss (plus bas = mieux)
    if pd.notna(ax) and pd.notna(ay):
        if ax > ay + auc_eps:
            return row["var_y"]  # drop y (x meilleur)
        elif ay > ax + auc_eps:
            return row["var_x"]  # drop x (y meilleur)
        else:
            # AUC ~ égalité -> compare logloss si dispo
            if pd.notna(lx) and pd.notna(ly):
                if lx + ll_eps < ly:
                    return row["var_y"]
                elif ly + ll_eps < lx:
                    return row["var_x"]
            # Sinon, compare propreté puis simplicité
            if pd.notna(nx) and pd.notna(ny) and nx != ny:
                return row["var_x"] if nx > ny else row["var_y"]  # drop celle qui a + de NA
            if pd.notna(cx) and pd.notna(cy) and cx != cy:
                return row["var_x"] if cx > cy else row["var_y"]  # drop la + complexe (cardinalité)
            # Par défaut: drop var_y
            return row["var_y"]

    # Si AUC manquante d'un côté, garde celle qui a une AUC non nulle
    if pd.notna(ax) and pd.isna(ay):
        return row["var_y"]
    if pd.isna(ax) and pd.notna(ay):
        return row["var_x"]

    # Dernier recours : cardinalité / NA
    if pd.notna(cx) and pd.notna(cy) and cx != cy:
        return row["var_x"] if cx > cy else row["var_y"]
    if pd.notna(nx) and pd.notna(ny) and nx != ny:
        return row["var_x"] if nx > ny else row["var_y"]

    return row["var_y"]

pv["drop_reco"] = pv.apply(choose_drop, axis=1)

# =========================
# 3) Liste finale des drops (règles + reco)
# =========================
to_drop = set(initial_drop) | set(pv["drop_reco"].dropna().tolist())

sorted(to_drop)


In [ ]:
pair_var = pair_var.merge(univar_scores, left_on="var_x", right_on="variable", how="left") \
                   .rename(columns={"auc":"auc_x", "logloss":"logloss_x"}) \
                   .drop(columns="variable")

pair_var = pair_var.merge(univar_scores, left_on="var_y", right_on="variable", how="left") \
                   .rename(columns={"auc":"auc_y", "logloss":"logloss_y"}) \
                   .drop(columns="variable")
